<center><h1>Section 5-2A Output Graphics</h1></center>

In [ ]:
import os
import sys
import matplotlib.pyplot as plt
from IPython.display import display, HTML

root_directory = os.path.dirname(os.path.dirname(os.getcwd()))

if os.path.join(root_directory, 'src') not in sys.path:
    sys.path.append(os.path.join(root_directory, 'src'))

from graphics_renderer import GraphicsRenderer

%matplotlib widget
%matplotlib inline

In [ ]:
HTML("""
<style>
.jp-RenderedImage {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

In [ ]:
gr = GraphicsRenderer('RESULTS5-2A-Test-0.0.0.json')

In [ ]:
df, msg = gr.render_section_5_2a_table_b_8_1()
display(HTML(df.to_html(index=False)))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14,8))
gr.render_section_5_2a_figure_b_8_9(fig, ax)
plt.show();